In [10]:
import torch

print("PyTorch 버전:", torch.__version__)
print("MPS 사용 가능:", torch.backends.mps.is_available())
print("MPS 준비 완료:", torch.backends.mps.is_built())

PyTorch 버전: 2.1.0
MPS 사용 가능: True
MPS 준비 완료: True


## 학습시킬 원천 데이터 불러오기 

In [2]:
import os
import json

모든 json 파일을 불러오는 함수

In [3]:
def load_training_raw_documents(training_root):
    data_list = []

    for folder_name in os.listdir(training_root): # 지정한 경로 안에 있는 모든 파일/폴더 이름을 리스트로 변환
        folder_path = os.path.join(training_root, folder_name) # join 메소드는 결국에 path를 만들어주는 메소드

        #여기서 folder_path 는 경로를 가지고 있다.


        if not os.path.isdir(folder_path):
            continue

        # 판례인지 심결례인지 구분
        if "판례" in folder_name:
            doc_type = "판례"
        elif "심결례" in folder_name:
            doc_type = "심결례"
        else:
            doc_type = "기타"

        # 사건유형: 민사, 형사 등
        # 폴더명이 "TS_1.판례_01.민사" 라면 ["TS", "1.판례", "01.민사"]로 쪼개짐
        case_type = folder_name.split("_")[-1]

        for file_name in os.listdir(folder_path): #여기서 file_name은 각각의 json파일을 담고있음
            if file_name.endswith(".json"):
                file_path = os.path.join(folder_path, file_name)  #이제 file_path는 문서의 full 절대경로를 가지게 된다. 

                #파일 읽기 
                try:
                    with open(file_path, "r", encoding="utf-8") as f:  #with문을 쓰면 파일을 자동으로 열고, 사용이 끝나면 자동으로 닫아줌
                        doc = json.load(f)  #불러온 파일 객체를 f 로 사용함

                        #여기서 doc은 딕셔너리 타입이다. json에 원래 있던 key-value 값을 가지는 딕셔너리. 

                        # 메타데이터 추가
                        doc["__file_path"] = file_path
                        doc["__doc_type"] = doc_type
                        doc["__case_type"] = case_type

                        data_list.append(doc)
                except Exception as e:
                    print(f"오류 발생: {file_path} → {e}")

    print(f"총 {len(data_list)}개의 원천 문서를 불러왔습니다.")
    return data_list


원천데이터 폴더안에 있는 모든 json데이터 딕셔너리 형태로 불러오기

In [4]:
training_path = "/Users/joonwon/Desktop/inha/4-1/rag/dataset/Training/01.원천데이터"
raw_documents = load_training_raw_documents(training_path)

총 53209개의 원천 문서를 불러왔습니다.


불러온 데이터 확인

In [5]:
print(raw_documents[0])

{'행정심판재결례일련번호': 238791, '사건명': '서지사항보정서반려처분취소청구', '사건번호': '2004-08324', '의결일자': '2004-09-27', '처분일자': None, '처분청': None, '재결청': '국민권익위원회', '재결구분명': None, '재결구분코드': None, '행정심판례상세링크': '/DRF/lawService.do?OC=pwshoot&target=decc&ID=238791&type=HTML&mobileYn=', '재결요지': '사 건 04-08324 서지사항보정서반려처분취소청구\n청 구 인 ○○ 인터내셔널.INC .\n(○○International, INC.)\n미국 조지아 ○-○ ○○ ○○ 스트리트 1940\n(1940 ○○ Street, ○○, Georgia ○-○, USA)\n대리인 변리사 박 ○ ○\n피청구인 특허청장\n청구인이 2004. 6. 10. 제기한 심판청구에 대하여 2004년도 제36회 국무총리행정심판위원회는 주문과 같이 의결한다.', '주문': '청구인의 청구를 기각한다.', '청구취지': '피청구인이 2004. 4. 23. 청구인에 대하여 한 서지사항보정서반려처분은 이를 취소한다.', '이유': "1. 사건개요\n청구인은 2002. 7. 10. '보조버너 또는 란스를 위한 장착설비'에 관하여 미국특허청을 수리관청으로, 대한민국 등을 지정국으로 하여 국제출원을 하고 미국 특허출원 '제US 09/902, 135호'에 대한 우선권을 주장한 후 2004. 1. 9. 대한민국에서 특허절차를 진행하기 위하여 피청구인에게 특허법 제203조의 규정에 의한 서면을 제출하면서 우선권번호를 위 '제US 09/902, 135호'로 기재하여 제출하였으나, 그 후 2004. 2. 20. 위 번호를 '제US 09/902, 139호'로 정정하여 달라는 내용의 서지사항보정서를 청구인에게 제출하였고, 이에 대하여 피청구인은 2004. 4. 23. 위 보정서가 법정기간을 경과하여 제출되었다는 이유로 반려(이하 '이 건 처분'이

In [6]:
print(raw_documents[0].keys())

dict_keys(['행정심판재결례일련번호', '사건명', '사건번호', '의결일자', '처분일자', '처분청', '재결청', '재결구분명', '재결구분코드', '행정심판례상세링크', '재결요지', '주문', '청구취지', '이유', '__file_path', '__doc_type', '__case_type'])


In [7]:
from collections import Counter

case_counter = Counter(doc["__case_type"] for doc in raw_documents)  #모든 문서에서 case_type값을 하나씩 센다.
print("사건유형별 문서 수:")
for k, v in case_counter.items(): #items()는 딕셔너리같은 자료구조에서 k,v 쌍을 튜플로 꺼내주는 함수
    print(f"{k}: {v}건")


사건유형별 문서 수:
08.특허.저작권: 2407건
10.개인정보.ICT: 1603건
06.기업: 1712건
09.금융조세: 2433건
01.민사: 13623건
05.행정: 17070건
02.가사: 1655건
07.근로자: 2408건
04.형사B(일반형): 5491건
03.형사A(생활형): 4807건


데이터 저장

In [17]:
import pickle

with open("raw_documents.pkl", "wb") as f:
    pickle.dump(raw_documents, f)
